In [1]:
import os

In [2]:
%pwd

'd:\\py4e\\Machine_Learning_from_scratch\\Machine Learning\\Industry Ready Projects\\Wine-Quality-Prediction-MLOPS-with-MLflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\py4e\\Machine_Learning_from_scratch\\Machine Learning\\Industry Ready Projects\\Wine-Quality-Prediction-MLOPS-with-MLflow'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/yashwanthreddy7178/Wine-Quality-Prediction-MLOPS-with-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="yashwanthreddy7178"
os.environ["MLFLOW_TRACKING_PASSWORD"]="2f0a09218b79018bb56442916a9544eb88a54c20"

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/yashwanthreddy7178/Wine-Quality-Prediction-MLOPS-with-MLflow.mlflow",
           
        )

        return model_evaluation_config


In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [20]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    print(model_evaluation_config)  # print the configuration values
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-02-04 18:27:00,644: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-04 18:27:00,649: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-04 18:27:00,659: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-04 18:27:00,662: INFO: common: created directory at: artifacts]
[2024-02-04 18:27:00,667: INFO: common: created directory at: artifacts/model_evaluation]
ModelEvaluationConfig(root_dir='artifacts/model_evaluation', test_data_path='artifacts/data_transformation/test.csv', model_path='artifacts/model_trainer/model.joblib', all_params=ConfigBox({'alpha': 0.2, 'l1_ratio': 0.1}), metric_file_name='artifacts/model_evaluation/metrics.json', target_column='quality', mlflow_uri='https://dagshub.com/yashwanthreddy7178/Wine-Quality-Prediction-MLOPS-with-MLflow.mlflow')


RestException: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}

In [18]:
import mlflow

mlflow.set_tracking_uri('https://dagshub.com/yashwanthreddy7178/Wine-Quality-Prediction-MLOPS-with-MLflow.mlflow')

with mlflow.start_run():
    mlflow.log_param("test_param", "test_value")

RestException: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}

In [14]:
import os

print(os.getenv('MLFLOW_TRACKING_URI'))

https://dagshub.com/yashwanthreddy7178/Wine-Quality-Prediction-MLOPS-with-MLflow.mlflow


In [15]:
os.environ.pop('MLFLOW_TRACKING_URI', None)

'https://dagshub.com/yashwanthreddy7178/Wine-Quality-Prediction-MLOPS-with-MLflow.mlflow'

In [19]:
import mlflow

mlflow.start_run()
mlflow.log_param("test_param", "test_value")
mlflow.end_run()

RestException: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}